## https://github.com/koito19960406/Studio
## flickr観光特性
https://www.jstage.jst.go.jp/article/aija/84/755/84_187/_pdf/-char/ja
## flikr api
https://www.flickr.com/services/api/
https://www.flickr.com/groups/api/

In [1]:
import flickrapi
from lxml import etree
from lxml.html.soupparser import fromstring
from lxml.etree import tostring
import bs4 as bs#HTML や XML から狙ったデータを抽出するためのライブラリ
import pandas as pd
from tqdm import tqdm#進捗状況や処理状況をプログレスバーで表示
import time
# import api keys and secrets
#from flickr_apikey import api_key_1, secret_1, api_key_2, secret_2, api_key_3, secret_3, api_key_4, secret_4

### PythonでFlickr APIから画像取得
https://qiita.com/hiraku00/items/dbdaad45ea54a35e51a4
### pythonの「tqdm」って何？簡単な例を用いて使い方を紹介
https://vector-ium.com/python-tqdm/
### 10分で理解する Beautiful Soup(scrapyによるクローラーなど)
https://qiita.com/Chanmoro/items/db51658b073acddea4ac
### PythonでWebスクレイピングする時の知見をまとめておく
https://vaaaaaanquish.hatenablog.com/entry/2017/06/25/202924
### 【実習編】非専門家のためのQGIS ～白地図を描こう_都道府県界～
https://note.com/kinari_iro/n/nfee9bc97b6d7

In [2]:
import os
# %reload_ext dotenv
# %dotenv
flickr_key = ""
flickr_sec = ""

key = flickr_key
sec = flickr_sec

### https://www.gsi.go.jp/KOKUJYOHO/CENTER/kendata/zenken.pdf

In [3]:
def get_photo(page, per_page):
    result=flickr.photos.search(
                            api_key=key,
#                            tags=tags,#取得したい写真のタグ
                            bbox='122.5557,24.0244,132.1038,34.4339',#'134.1509,33.2559,136.5915,35.4645',#'##'103.6920359,1.1304753,104.0120359,1.4504753',#mini_long,mini_lati,max_long,max_lati
                            accuracy=16,#the highest accruacy of location　information: Street is ~16
                            has_geo=1,# Any photo that has been geotagged
                            geo_context=2, # 0,not defined ;2, outdoor.
                            content_type=1, # includes only photos
                            per_page=per_page,
                            page=page
                            )
    return result

In [4]:
def get_location(list):
    # Getting location data
    list_lat=[]#緯度のリスト
    list_lon=[]#経度のリスト
    api_condition=0
    for id in tqdm(list, desc='location',position=0, leave=True):
        result_location=flickr.photos.geo.getLocation(
            api_key=key,
            photo_id=id)
        str_result_location=tostring(
            result_location, 
            pretty_print=True).strip()
        soup=bs.BeautifulSoup(
            str_result_location,'lxml')
        latitude=soup.find('location')['latitude']
        longitude=soup.find('location')['longitude']
        list_lat.append(latitude)
        list_lon.append(longitude)
    return list_lat, list_lon

In [5]:
def d_flickr(api_key_1, secret_1):#, tags):
#     for tag in tags:
        df_combined=pd.DataFrame()
#         # for grid in tqdm(list_grid_combined,desc='inner_loop',position=0):
        page=1
#         pbar = tqdm(total=100,desc='outer_loop',position=1, leave=False)

        
        while page <=10:
            
            # ここでget_photoを動かす
            
            #get photo in lxml formt
            result=get_photo(page,250)

            # parse the result to a string and convert it a soup object
            str_result=tostring(result, pretty_print=True).strip()#tostringで整形
            soup=bs.BeautifulSoup(str_result,'lxml')#HTMLの構造解析をした上で整形、lxmlは解析器
            photos=soup.find_all('photo') # find all the photos
            if len(photos)==0:
                print("photo end")
                break
            list_id=[]#idリスト
            list_title=[]#titleリスト

            # store the id and title of each photo as lists
            for photo in photos:
                id=photo['id']
                title=photo['title']
                list_id.append(id)
                list_title.append(title)

            # convert lists to a dataframe
            dictionary_photo={'id': list_id, 'title': list_title}
            df_photos=pd.DataFrame.from_dict(dictionary_photo)

            # ここでget_locationを動かす
            
            #get location
            list_lat, list_lon=get_location(list_id)#上記のidリストが引数

            # add lists as columns to dataframe
            df_photos['latitude']=list_lat
            df_photos['longitude']=list_lon

            # append df_photos to df_combined
            df_combined=df_combined.append(df_photos, ignore_index=True)

            # update page number
            page+=1

            # update the progress bar  
#            pbar.update(100)

#        pbar.close()
        # save the result to a csv file
        df_combined.to_csv('kyushu.csv')#_'+tag+'.csv')

In [6]:
if __name__ =='__main__':
    # tags
#     tags=[
#         'nature',
#         'leisure',
#         'waterfront',
#         'relax',
#         'chill',
#         'history'
#         ]

    wait_time = 1
    flickr = flickrapi.FlickrAPI(key,sec)

    
    # run the function
    d_flickr(key, sec)#, tags)

location:  35%|███████████▏                    | 87/250 [02:08<03:21,  1.23s/it]do_request: Status code 500 received, content:
    <!DOCTYPE html>
<html xmlns:cc="http://creativecommons.org/ns#" lang="en-us" class="no-js fluid html-error-500-page-view maximized-layout">
<head>
	<meta property="fb:app_id" content="137206539707334" />
	<meta property="og:site_name" content="Flickr" />
	
	<meta property="og:updated_time" content="2021-09-14T05:07:43.413Z" />
	
	<script type="application/ld+json">
		[{
			"@context": "http://schema.org",
			"@type": "WebSite",
			"name": "Flickr",
			"url": "https://www.flickr.com",
			"potentialAction": {
				"@type": "SearchAction",
				"target": "https://www.flickr.com/search?text={search_term_string}
    structured=yes",
				"query-input": "required name=search_term_string"
			}
		},
		{
			"@context": "http://schema.org",
			"@type": "Organization",
			"url": "http://www.flickr.com",
			"logo": "https://www.flickr.com/images/opensearch-flickr-logo.png

    #x27;unsafe-eval
    #x27; 
    #x27;unsafe-inline
    #x27; 
    #x27;nonce-28a763f87f934ef96ed3455cc8c1664c
    #x27; https://*.flickr.com https://*.staticflickr.com https://cdn.yahooapis.com https://yui-s.yahooapis.com https://www.googletagmanager.com https://assets.adobedtm.com https://*.2o7.net https://api.digitalriver.com https://cdn.siftscience.com https://*.flickrpro.com https://securepubads.g.doubleclick.net https://adservice.google.com https://cdn.ampproject.org https://*.google.com https://*.google-analytics.com https://*.googleadservices.com https://*.doubleclick.com https://*.doubleclick.de https://*.doubleclick.net https://*.googletagservices.com https://*.googleadservices.com https://*.googlesyndication.com https://*.googleapis.com https://boards.greenhouse.io https://adserver.adtech.advertising.com https://cdn.jsdelivr.net https://js.stripe.com https://c2shb.ssp.yahoo.com https://prebid.a-mo.net https://*.trustarc.com https://trustarc.mgr.consensu.org https://shb.ri

    
    s.body
    
    (s.body.lastChild
    
    s.body.lastChild.className
    
    "wipe-msg"===s.body.lastChild.className||(a=t.replace("{url}",w).replace("{faq}",z),"number"==typeof l.innerWidth?n=l.innerWidth:s.body.clientWidth
    
    (n=s.body.clientWidth),n
    
    n<162
    
    (e=n,o="10px"),r='<div class="wipe-msg" style="font-size:'+o+';text-align:left;"><div style="margin-bottom:3px;"><img alt="Flickr" width="'+e+'" src="https://combo.staticflickr.com/pw/images/logo_home.png"></div><div style="padding-left:5px;line-height:1.2em;">'+a+"</div></div>",s.body.style.margin="0",s.body.innerHTML=r))},200)}function a(){for(var e=self,o=0;e!==e.parent;)o+=1,e=e.parent;return o}var i,c,l=e,s=o,d=top.location,m=self.location,g=s.referrer,p=/^[A-Za-z]+:\/{2,3}(?:[0-9\-A-Za-z]+\.)*flickr\.(?:(?:com)|(?:net))(?::\d+)?(?:\/[^?#]*)?(?:\?[^#]*)?(?:#.*)?$/i,h=m.protocol
    
    m.protocol.indexOf("http")<0,f=p.test(m),b=f?m.href:n+"/",u=f
    
    /\/photos\/[^\/]+\/(\d+)/i.exec(m.pa

    ","root":"yui:3.16.0\u002F","filter":"min","logLevel":"error","combine":true,"patches":[function patchOptionalRequires(Y, loader) {
    var getRequires = loader.getRequires,
        addModule = loader.addModule;
    // patching `addModule` method to support polyfills
    loader.addModule = function (mod) {
        var configFn = mod 
    
     mod.configFn;
        if (mod 
    
     mod.test) {
            mod.configFn = function (mod) {
                if (!mod.test(Y)) {
                    // if a test fails, the module should be dropped from the registry
                    return false;
                }
                if (configFn) {
                    // falling back to the original configFn if the test passed
                    return configFn.apply(this, arguments);
                }
            };
        }
        return addModule.apply(this, arguments);
    };
    // patching `getRequires` to support optional requires
    loader.getRequires = function (mod) {
      

    select
    utm_campaign=flickr-launch-15
    utm_source=Flickr
    utm_medium=display
    utm_content=join-300x250","photopagelink":"\u002Faccount\u002Fupgrade\u002Fpro?coupon=FLICKRPRO15
    select
    utm_campaign=flickr-launch-15
    utm_source=Flickr
    utm_medium=display
    utm_content=join-300x250"}},"positions":{"billboard":[{"desc":"feed_billboard","name":"\u002F21833886396\u002FFlickr_FeedBillboard_Desktop_970x250","ad_unit_desktop":"\u002F21833886396\u002FFlickr_Feed_Desktop_Leaderboard","ad_unit_mobile":"\u002F21833886396\u002FFlickr_Feed_Mobile_Leaderboard"}],"feed":[{"desc":"feed_card_A","name":"\u002F21833886396\u002FFlickr_FeedCard_A_Desktop_300x250","ad_unit_desktop":"\u002F21833886396\u002FFlickr_FeedCard_A_Desktop_300x250","ad_unit_mobile":"\u002F21833886396\u002FFlickr_FeedCard_A_Mobile_300x250"},{"desc":"feed_card_B","name":"\u002F21833886396\u002FFlickr_FeedCard_B_Desktop_300x250","ad_unit_desktop":"\u002F21833886396\u002FFlickr_FeedCard_B_Desktop_300x250","a

    
     typeof YUI == "undefined") { throw new Error("YUI was not injected correctly!"); } self.YUI = self.YUI || YUI; for (i = 0; i < l; i++) { callback.shift()(); } } function decrementRequestPending() { self._pending--; if (self._pending <= 0) { setTimeout(flush, 0); } else { load(); } } function createScriptNode(src) { var node = d.createElement('script'); if (node.async) { node.async = false; } if (ie) { node.onreadystatechange = function () { if (/loaded|complete/.test(this.readyState)) { this.onreadystatechange = null; decrementRequestPending(); } }; } else { node.onload = node.onerror = decrementRequestPending; } node.setAttribute('src', src); return node; } function load() { if (!config.seed) { throw new Error('YUI_config.seed array is required.'); } var seed = config.seed, l = seed.length, i, node; if (!self._injected) { self._injected = true; self._pending = seed.length; } for (i = 0; i < l; i++) { node = createScriptNode(seed.shift()); head.appendChild(node); if (node.asy

    title="+escape(t),r=function(){for(var e=!1,t=0;t<n.length;t++){try{e=n[t]()}catch(e){continue}break}return e}();if(!r)return;r.open("GET","https://www.flickr.com/beacon_flanal_reboot_event.gne?"+c,!0),r.withCredentials=!0,r.setRequestHeader("Content-type","application/x-www-form-urlencoded"),r.send()}catch(e){}}}(window,document);
	</script>	<script nonce="28a763f87f934ef96ed3455cc8c1664c" src="//assets.adobedtm.com/launch-ENd8846597b4254546afff313cb328c80d.min.js" async></script>
	<script type="text/javascript" nonce="28a763f87f934ef96ed3455cc8c1664c">
	!function(o,w){!function(o,w){o.snowplow||(o.GlobalSnowplowNamespace=o.GlobalSnowplowNamespace||[],o.GlobalSnowplowNamespace.push("snowplow"),o.snowplow=function(){(o.snowplow.q=o.snowplow.q||[]).push(arguments)},o.snowplow.q=o.snowplow.q||[])}(o)}(window,document);
	</script>	
	<script class='modelExport' type="text/javascript" nonce="28a763f87f934ef96ed3455cc8c1664c">
	
		var comboRetry = 0,	
		    initOk = false,	
		    clientA

    
     ex.message ? ex.message : ex), window.location.href, ex);
					} else {
						sendRequest('flickr.hermes.clientapp.initfail', 'client app init fail');
						beaconError('Client app failure: ' + (ex 
    
     ex.message ? ex.message : ex), window.location.href, ex);
		
					}
				} catch (e) {
					sendRequest('flickr.hermes.clientapp.initfail', 'client app init fail');
					beaconError('Client app failure: ' + (ex 
    
     ex.message ? ex.message : ex), window.location.href, ex);
		
				}
		
			});
		};
	
		var useSuccess = function (Y, status) {
		
			if (status 
    
     !status.success) {
		
				if (comboRetry === 0) {
					comboRetry++;
		
					if (typeof console !== 'undefined' 
    
     console.warn) {
						console.warn('There was an error loading client modules, retrying: ' + status.msg);
					}
		
					if (sendRequest) {
						sendRequest('flickr.hermes.clientapp.moduleretry', 'client app module loading fail, retrying');
					}
		
					Y.use('client-app', 'error

FlickrError: do_request: Status code 500 received

In [38]:
pip install folium

     |████████████████████████████████| 94 kB 917 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


## https://qiita.com/kanakona/items/9e67b60b4ae3966c49fa

In [49]:
import folium
df = pd.read_csv('kyushu.csv')#, encoding='shift_jis')
df.head(10)

,Unnamed: 0,id,title,latitude,longitude
0,0,51242052977,NaN,24.441816,124.118813
1,1,51241928904,Podmořská,33.258861,131.535613
2,2,51241154886,Obklopena krásou,33.258861,131.535613
3,3,51240434992,Srdečné veselí,33.258861,131.535613
4,4,51233848833,Iris Flower Festival 2021,33.779967,130.486195
5,5,51228864156,Iris Flower Festival 2021,33.779967,130.486195
6,6,51227008535,Watatsumi Shrine,33.705902,130.416684
7,7,51212681709,NaN,24.339041,124.158202
8,8,51211904216,SORA the sky 2021.05.02.,33.767250,130.470639
9,9,51210343891,B/W #1557,33.575459,130.400223


In [5]:
# 地図オブジェクト作成
map = folium.Map(location=[35.710402, 139.810668], zoom_start=18)

# ポップアップの作成(「show=True」で常に表示)
p_up = folium.Popup('東京スカイツリー', min_width=0, max_width=1000, show=True)

# 地図オブジェクトにプロット
folium.Marker(location=[35.710402, 139.810668], popup=p_up).add_to(map)

# 地図表示
map

In [50]:
from folium.plugins import HeatMap
# 地図オブジェクト作成
map = folium.Map(location=[35.710402, 139.810668], zoom_start=5)

# ヒートマップレイヤを地図オブジェクトに追加
HeatMap(df[['latitude', 'longitude']], radius=5, blur=5).add_to(map)

# 地図表示
map